In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

In [2]:
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [3]:
def get_error(pred, y):
    return np.sum((pred != y).astype(int)) / len(y)

In [4]:
def adaboost(X, y, N):

    n_objects = len(X)

    n_classes = len(np.unique((y)))

    w = np.ones(n_objects) / n_objects

    models = []

    for n in range(N):
        clf = DecisionTreeClassifier(max_depth=1)
        clf.fit(X, y, w)

        predictions = clf.predict(X)
        e = get_error(predictions, y)
        if e >= 1 - 1/n_classes: 
            break

        alpha = 0.5 * np.log((1 - e) / e)

        match = predictions == y

        w[np.logical_not(match)] *= np.exp(alpha)
        w[match] *= np.exp(-alpha)

        w /= w.sum()

        models.append((alpha, clf))
    
    return models

In [5]:
def predict(X, models):
    
    n_classes = 3
    n_objects = len(X)
    
    y_pred = np.zeros((n_objects, n_classes))
    
    for alpha, clf in models:
        prediction = clf.predict(X)
        y_pred[range(n_objects), prediction] += alpha
    
    y_pred = np.argmax(y_pred, axis=1)
    
    return y_pred

In [6]:
N = 50

models = adaboost(X_train, y_train, N)

In [7]:
# Для начала отмасштабируем выборку
X_ = X_train.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [8]:
X.shape

(178, 13)

In [9]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(13,1), eig_pairs[1][1].reshape(13,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[-0.1017576   0.48681992]
 [ 0.2480939   0.22964871]
 [ 0.00680322  0.35592092]
 [ 0.23862015 -0.00568027]
 [-0.11463709  0.33557152]
 [-0.3914133   0.07898993]
 [-0.42378901  0.03598345]
 [ 0.28759444  0.02028919]
 [-0.3084663   0.02078229]
 [ 0.14324402  0.48304911]
 [-0.32503975 -0.21491215]
 [-0.39047192 -0.11659715]
 [-0.26204152  0.41164461]]


In [10]:
new_X_train=X_.dot(W)

In [11]:
new_X_test=X_test[:, :]

In [12]:
# центрирование - вычитание из каждого значения среднего по строке
for j in range(cols):
    new_X_test[:, j] -= means[j]

# деление каждого значения на стандартное отклонение
for i in range(cols):
    new_X_test[:, i] /= std[i]

In [13]:
new_X_test=new_X_test.dot(W)

In [14]:
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [15]:
N = 50 

models = adaboost(new_X_train, y_train, N)

In [16]:
print(f'Точность алгоритма на обучающей выборке: {(1 - get_error(predict(new_X_train, models), y_train)) * 100:.3f}')

Точность алгоритма на обучающей выборке: 96.992


In [17]:
print(f'Точность алгоритма на тестовой выборке: {(1 - get_error(predict(new_X_test, models), y_test)) * 100:.3f}')

Точность алгоритма на тестовой выборке: 88.889
